In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import pandas as pd
import glob
import os

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from matplotlib.colors import LinearSegmentedColormap#, ListedColormap, Normalize, TwoSlopeNorm

from bokeh.models import LinearColorMapper

import holoviews as hv
from holoviews import opts
from holoviews.operation.datashader import rasterize as hds_rasterize

import geoviews.feature as gf # only needed for coastlines
#from geoviews.operation import resample_geometry
import geoviews as gv
#from datashader.mpl_ext import dsshow, alpha_colormap
#import geoviews.tile_sources as gts

#import geocat.datafiles as gdf  # Only for reading-in datasets
from xarray import open_mfdataset
import xarray as xr

import metpy.calc as mpcalc
from metpy.units import units

from mpas_tools.database.utils import choose_forecast
from mpas_tools.plotting.utils import nonlinear_colorbar
from mpas_tools.utils.tools import find_TC_bbox
from mpas_tools.datashader_tools.utils import datashader_wrapper

gv.extension("bokeh","matplotlib")

opts.defaults(
    opts.Image(width=1200, height=600),
    opts.RGB(width=1200, height=600))

#hv.output(dpi=300, fig='png')

In [ ]:
hires_folder = r"/gpfs/group/cmz5202/default/cnd5285/MPAS_3km"
hires_files = glob.glob(os.path.join(hires_folder, '*.cam.h0.*0.nc'), recursive=False)
hires_files.sort
# hires_0830_init = [f for f in hires_files if '08-30' in f]
# hires_0831_init = [f for f in hires_files if '08-31' in f]
# hires_0901_init = [f for f in hires_files if '09-01' in f]
# hires_0902_init = [f for f in hires_files if '09-02' in f]

hires_mesh_file = os.path.join(hires_folder, 'x20.835586.florida.init.CAM.nc')

In [ ]:
# hires_0830_ds = xr.open_mfdataset(hires_0830_init)
# hires_0831_ds = xr.open_mfdataset(hires_0831_init)
# hires_0901_ds = xr.open_mfdataset(hires_0901_init)
# hires_0902_ds = xr.open_mfdataset(hires_0902_init)

hires_ds = xr.open_mfdataset(hires_files)
hires_mesh = xr.open_dataset(hires_mesh_file, decode_times=False)

In [ ]:
hires_ds['maxU10'] = hires_ds['U10'].max(dim='time')
hires_ds['TC_U10'] = hires_ds['maxU10'].where(hires_ds['maxU10'] >= 18)
hires_ds['PRECIP_T'] = hires_ds['PRECT'].sum(dim='time')
hires_ds

In [ ]:
hires_mesh

In [ ]:
hires_0831_ds['U10'].values.max()

In [ ]:
hires_0901_ds

In [ ]:
print(len(hires_0830_ds['time'].values))
print(len(hires_0831_ds['time'].values))
print(len(hires_0901_ds['time'].values))
print(len(hires_0902_ds['time'].values))

# 73 hour/3 day trek

In [ ]:
print(hires_ds['FLUT'].isel(time=[41]).values.min())
print(hires_ds['FLUT'].isel(time=[41]).values.max())

In [ ]:
print(hires_ds['U10'].isel(time=[41]).values.max())

In [ ]:
# Remake of geonetcast colorbar (same as imported colormap)

# custom_colors = [
#                 (0, '#ffff00'), 
#                 (0.25, '#002266'), 
#                 (1, '#002266')]
bw_colors = [(0, '#000000'), ((46/153), '#000000'), ((1-(7/153)), '#ffffff'), (1, '#ffffff')]
#bw_colors = [(0, '#000000'), (1, '#ffffff')]  
bw_cmp = LinearSegmentedColormap.from_list('FLUT bw', bw_colors, N=153)


cyan_colors = [(0, '#00ffff'), # cyan
               ((21/73), '#000073'), # navy
               ((22/73), '#000073'), # navy
               ((35/73), '#00ff00'), # lime green
               ((45/73), '#ffff00'), # yellow
               ((55/73), '#ff0000'), # red
               ((65/73), '#000000'), # black
               (1, '#e6e6e6')] # dark white

cyan_cmp = LinearSegmentedColormap.from_list('FLUT cyan', cyan_colors, N=73)
pink_colors = [(0, '#ff7fcb'), (1, '#7f007f')]
pink_cmp = LinearSegmentedColormap.from_list('FLUT pink', pink_colors, N=11)
white_cmp = LinearSegmentedColormap.from_list('FLUT white', [(0, '#ffffff'), (1, '#ffffff')], N=19)

#colors = np.vstack((bw_colors, cyan_colors, pink_colors, [(0, '#ffffff'), (1, '#ffffff')]
#plt.register_cmap(cmap=bw_cmp)
#plt.register_cmap(cmap=cyan_cmp)
#plt.register_cmap(cmap=pink_cmp)
#plt.register_cmap(cmap=white_cmp)

bws = plt.get_cmap(bw_cmp)
bws_colors = bws(np.linspace(0, 1, 153))
cyans = plt.get_cmap(cyan_cmp)
c_colors = cyans(np.linspace(0, 1, 73))
pinks = plt.get_cmap(pink_cmp)
p_colors = pinks(np.linspace(0, 1, 11))
whites = plt.get_cmap(white_cmp)
w_colors = whites(np.linspace(0, 1, 19))

all_colors = np.vstack((bws_colors, c_colors, p_colors, w_colors))
flut_gnc = LinearSegmentedColormap.from_list('FLUT GeoNetCast', all_colors)
flut_gnc.reversed()

In [ ]:
# Weathernerds.org scale

bw_colors = [(0, '#000000'), (1, '#BCBCBC')]  
bw_cmp = LinearSegmentedColormap.from_list('FLUT bw', bw_colors, N=70)

rainbow_colors = [(0, '#00ffff'), # cyan
               ((12/70), '#000073'), # navy
               ((25/70), '#00ff00'), # lime green
               ((32/70), '#ffff00'), # yellow
               ((42/70), '#ff0000'), # red
               ((52/70), '#000000'), # black
               ((59/70), '#e6e6e6'), # dark white
               (1, '#7f007f')] # purple


rainbow_cmp = LinearSegmentedColormap.from_list('FLUT colors', rainbow_colors, N=70)

bws = plt.get_cmap(bw_cmp)
bws_colors = bws(np.linspace(0, 1, 128))
rainbow = plt.get_cmap(rainbow_cmp)
r_colors = rainbow(np.linspace(0, 1, 128))

all_colors = np.vstack((bws_colors, r_colors))
flut_wn = LinearSegmentedColormap.from_list('FLUT WeatherNerds', all_colors)
flut_wn.reversed()

In [ ]:
T_to_FLUT(60, 'C') - T_to_FLUT(55, 'C')

In [ ]:
# https://cimss.ssec.wisc.edu/satellite-blog/wp-content/uploads/sites/5/2017/09/GOES16_CleanWindow_Landfall-20170920_0957_1136anim.gif
def T_to_FLUT(T, unit='K'):
    if unit == 'C':
        T += 273.15
    sigma = 5.6693E-8
    olr = sigma*(T**4)
    
    return olr

# Normalized cimss scale

bw_colors = [(0, '#BCBCBC'), (1, '#000000')]  
#bw_cmp = LinearSegmentedColormap.from_list('FLUT bw', bw_colors, N=475)
bw_cmp = LinearSegmentedColormap.from_list('FLUT bw', bw_colors, N=435)

levels = np.array([T_to_FLUT(temp, 'C') for temp in [-110, -105, -87.5, -80, -70, -60, -50, -35, -27.5, -22.5]])
fracs = levels-T_to_FLUT(-110, 'C')

rainbow_colors = [(0, '#0febff'), # cyan
                  ((fracs[1]/fracs[-1]), '#7f007f'), # purple
                  ((fracs[2]/fracs[-1]), '#e5e4e5'), # white
                  ((fracs[3]/fracs[-1]), '#000000'), # black
                  ((fracs[4]/fracs[-1]), '#ff0000'), # red
                  ((fracs[5]/fracs[-1]), '#FFFF00'), # yellow
                  ((fracs[6]/fracs[-1]), '#00FF00'), # green
                  ((fracs[7]/fracs[-1]), '#000073'), # navy
                  (1, '#00ffff')] # cyan


rainbow_cmp = LinearSegmentedColormap.from_list('FLUT colors', rainbow_colors, N=184)

bws = plt.get_cmap(bw_cmp)
#bws_colors = bws(np.linspace(0, 0.7464215110803862, 100))
#bws_colors = bws(np.linspace(0, 1, 475))
bws_colors = bws(np.linspace(0, 1, 435))
rainbow = plt.get_cmap(rainbow_cmp)
#r_colors = rainbow(np.linspace(0.19764283851590042, 1, 150))
r_colors = rainbow(np.linspace(0, 1, 184))

all_colors = np.vstack((r_colors, bws_colors))
flut_cimss = LinearSegmentedColormap.from_list('FLUT CIMSS', all_colors)
flut_cimss

In [ ]:
def FLUT_to_T(olr, unit='C'):
    sigma = 5.6693E-8
    T = (olr/sigma)**0.25
    if unit == 'C':
        T -= 273.15
    return T

print(FLUT_to_T(75))

In [ ]:
#hv.output(dpi=200, backend='bokeh', fig='png')
central_longitude = 0.0
projection = ccrs.Robinson(central_longitude=central_longitude)

target_var = 'FLUT'
target_time = 17
pixel_ratio = 5
title = f"0831 TS {target_time} {target_var} PR {pixel_ratio}"

lon_range, lat_range = find_TC_bbox(hires_0831_ds, 'north atlantic', target_time, center_dist=1000)
rasterized_0831 = datashader_wrapper(hires_mesh, hires_0831_ds, target_var, target_time, pixel_ratio=pixel_ratio, 
                                     lon_range=lon_range, lat_range=lat_range)
final = rasterized_0831.opts(tools=['hover'], colorbar=True, width=800,
                             title=title) * gf.coastline(projection=projection).options(scale='10m')
#hv.save(final, f'../datashader/output_imgs/{title}.png', backend='bokeh', dpi=300)

final

In [ ]:
#hv.output(dpi=200, backend='bokeh', fig='png')
hv.output(dpi=300, backend='matplotlib', fig='png')
central_longitude = 0.0
projection = ccrs.Robinson(central_longitude=central_longitude)
norm_wn = TwoSlopeNorm(vmin=T_to_FLUT(-110, 'C'), vmax=T_to_FLUT(60, 'C'), vcenter=T_to_FLUT(-22.5, 'C')) # for weathernerds
norm_gnc = TwoSlopeNorm(vmin=T_to_FLUT(170), vmax=T_to_FLUT(378), vcenter=T_to_FLUT(260)) #230 should be cyan



target_var = 'FLUT'
target_time = 17+24
pixel_ratio = 5
title = f"0831 TS {target_time} {target_var} PR {pixel_ratio}"

#cpt = loadCPT(r'../plotting/IR4AVHRR6.cpt')
#cpt_convert = LinearSegmentedColormap('cpt', cpt)
#norm = Normalize(vmax=500, vmin=0)
m_opts = dict(colorbar=True, cmap=flut_wn, title=title, norm=norm_wn)#, fig_size=(10, 10) doesn't work for some reason
lon_range, lat_range = find_TC_bbox(hires_ds, 'north atlantic', target_time, center_dist=1000)
rasterized_0831 = datashader_wrapper(hires_mesh, hires_ds, target_var, target_time, pixel_ratio=pixel_ratio, 
                                     lon_range=lon_range, lat_range=lat_range)
rasterized_0831.opts(**m_opts) * gf.coastline(projection=projection).options(scale='10m')

In [ ]:
T_to_FLUT(-70, 'C')

In [ ]:
temps = [-110, -80, -60, -40, -22.5, 55]
[print(T_to_FLUT(T, 'C')) for T in temps]

In [ ]:
['cyan', 'black', 'yellow', 'forest green', 'cyan', 'black']

In [ ]:
hv.output(dpi=200, backend='bokeh', fig='png')
#hv.output(dpi=300, backend='matplotlib', fig='png')
central_longitude = 0.0
projection = ccrs.Robinson(central_longitude=central_longitude)

target_var = 'FLUT'
target_time = 17+24
pixel_ratio = 5
#title = f"0831 TS {target_time} {target_var} PR {pixel_ratio}"
title = f'Upwelling Longwave Flux: {str(hires_ds["time"].values[target_time])} UTC'

lon_range, lat_range = find_TC_bbox(hires_ds, 'north atlantic zoomed', target_time, center_dist=1000)
rasterized_0831 = datashader_wrapper(hires_mesh, hires_ds, target_var, target_time, pixel_ratio=pixel_ratio, 
                                     lon_range=lon_range, lat_range=lat_range)
final = rasterized_0831.opts(tools=['hover'], colorbar=True, cmap=flut_cimss, width=800, title=title, 
                             clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C'))) *\
gf.coastline(projection=projection).options(scale='10m')
#hv.save(final, f'../datashader/output_imgs/{title}.png', backend='bokeh', dpi=300)

final

# Compare with obs image in side-by-side for presentation

In [ ]:
# omit white, just start at TS strength
colors = [(0, '#FFA500'), ((14/67), '#FFA500'), ((15/67), '#880808'), (1, '#880808')]
nhc_ws_cmp = LinearSegmentedColormap.from_list('NHC Categorical Wind Swath', colors, N=68)

nhc_ws_cmp

In [ ]:
# Categorical wind swath plot

hv.output(dpi=200, backend='bokeh', fig='png')
#hv.output(dpi=300, backend='matplotlib', fig='png')
central_longitude = 0.0
proj = ccrs.Robinson(central_longitude=central_longitude)
#proj = ccrs.GOOGLE_MERCATOR

target_var = 'TC_U10'
target_time = 17+24
pixel_ratio = 5
#title = f"0831 TS {target_time} {target_var} PR {pixel_ratio}"
title = f"10-Meter Wind Swaths"
g_scale = '10m'

lon_range, lat_range = find_TC_bbox(hires_ds, 'florida', target_time, center_dist=1000)
rasterized_all = datashader_wrapper(hires_mesh, hires_ds, target_var, target_time, 
                                    pixel_ratio=pixel_ratio, lon_range=lon_range, lat_range=lat_range)

#final = gts.EsriImagery(projection=proj, global_extent=False) *\
final = gf.ocean(fill_color='#e4f1fa', line_color='face', projection=proj).options(scale=g_scale) *\
gf.states(line_width=0.5, line_color='#616161', fill_color='#EBEBEB', projection=proj).options(scale=g_scale) *\
rasterized_all.opts(show_legend=True, colorbar=False, cmap=nhc_ws_cmp, clim=(18, 85), width=800, title=title) *\
gf.coastline(line_width=0.5, line_color='#323232', fill_color=None, projection=proj).options(scale=g_scale)
#hv.save(final, f'../datashader/output_imgs/{title}.png', backend='bokeh', dpi=300)

final

# Compare with obs image in side-by-side for presentation

In [ ]:
# Precip totals plot

hv.output(dpi=200, backend='bokeh', fig='png')
#hv.output(dpi=300, backend='matplotlib', fig='png')
central_longitude = 0.0
proj = ccrs.Robinson(central_longitude=central_longitude)
#proj = ccrs.GOOGLE_MERCATOR

target_var = 'PRECIP_T'
target_time = 17+24
pixel_ratio = 5
#title = f"0831 TS {target_time} {target_var} PR {pixel_ratio}"
title = f"Precipitation Totals Swaths"
g_scale = '10m'

lon_range, lat_range = find_TC_bbox(hires_ds, 'florida', target_time, center_dist=1000)
rasterized_all = datashader_wrapper(hires_mesh, hires_ds, target_var, target_time, 
                                    pixel_ratio=pixel_ratio, lon_range=lon_range, lat_range=lat_range)

#final = gts.EsriImagery(projection=proj, global_extent=False) *\
#final = gf.ocean(fill_color='#e4f1fa', line_color='face', projection=proj).options(scale=g_scale) *\
#gf.states(line_width=0.5, line_color='#616161', fill_color='#EBEBEB', projection=proj).options(scale=g_scale) *\
rasterized_all.opts(colorbar=True, width=800, title=title) *\
gf.coastline(line_width=0.5, line_color='#323232', fill_color=None, projection=proj).options(scale=g_scale)
#hv.save(final, f'../datashader/output_imgs/{title}.png', backend='bokeh', dpi=300)

final

In [ ]:
(rs[-1]*rs[-2]*rs[-3]*r[-4]*r[-5]*r[-6]*r[-7]*5) * gf.coastline(projection=projection).options(scale='10m')

In [ ]:
hv.output(dpi=200, backend='bokeh', fig='png')
#hv.output(dpi=300, backend='matplotlib', fig='png')
central_longitude = 0.0
projection = ccrs.Robinson(central_longitude=central_longitude)

target_var = 'FLUT'
target_time = 17+24
pixel_ratio = 5
#title = f"0831 TS {target_time} {target_var} PR {pixel_ratio}"
title = f"Upwelling Longwave Flux: {str(hires_ds['time'].values[target_time])} UTC"

lon_range, lat_range = find_TC_bbox(hires_ds, 'florida', target_time, center_dist=1000)
rasterized_all = datashader_wrapper(hires_mesh, hires_ds, target_var, target_time, pixel_ratio=pixel_ratio, 
                                     lon_range=lon_range, lat_range=lat_range)
rasterized_all = rasterized_all.redim.range(FLUT=(40, 700))
final = rasterized_all.opts(tools=['hover'], colorbar=True, cmap=flut_wn, width=800, zlim=(T_to_FLUT(-110), T_to_FLUT(60)),
                             title=title) * gf.coastline(projection=projection).options(scale='10m')
#hv.save(final, f'../datashader/output_imgs/{title}.png', backend='bokeh', dpi=300)

final

# Compare with obs image in side-by-side for presentation

In [ ]:
rasterized_all.opts(tools=['hover'], colorbar=True, cmap=flut_wn, width=800, zlim=(T_to_FLUT(-110), T_to_FLUT(60)),
                             title=title) * gf.coastline(projection=projection).options(scale='10m')

In [ ]:
def T_to_FLUT(T, unit='K'):
    if unit == 'C':
        T = T+273.15
    sigma = 5.6693E-8
    olr = sigma*(T**4)
    
    return olr

T_to_FLUT(-60, 'C')

In [ ]:
T = -20
T_K = T+273.15
T_K = 170
T_K = 378
sigma = 5.6693E-8
olr = sigma*(T_K**4)
olr

In [ ]:
opts?

In [ ]:
central_longitude = 0.0
projection = ccrs.Robinson(central_longitude=central_longitude)

target_var = 'FLUT'
print(f'Rasterizing data.')
#for i in range(2, 9):
#for i in range(9, 36):
#for i in range(36, 59):
#for i in range(43, 59):
for i in range(59, 73):
    time_UTC = str(hires_ds['time'].isel(time=[i]).values[0])
    fig_title = f'Upwelling Longwave Flux: {time_UTC} UTC'
    lon_range = (269.61516038043123, 289.55559738239776)
    lat_range = (16.810663650794172, 34.797070925284935)
    rasterized = datashader_wrapper(hires_mesh, hires_ds, target_var, i, pixel_ratio=10, lon_range=lon_range, lat_range=lat_range)
    final_img = rasterized.opts(colorbar=True, cmap=flut_cimss, toolbar=None, 
                                clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')), title=fig_title, width=800) * gf.coastline(projection=projection).options(scale='10m')

    print(f'Exporting image {i}.')
    hv.save(final_img, os.path.join('/storage/work/cnd5285/github/mpas_tools/mpas_tools/figs/flut_gif', f'FLUT_{time_UTC}.png'), 
            dpi=300, backend='bokeh', fmt='png', width=800)

#print(f'Time elapsed: {end-start} seconds.')

In [ ]:
output_folder = r"/storage/work/cnd5285/github/mpas_tools/mpas_tools/figs/flut_gif"
from PIL import Image
import glob
# Create the frames
frames = []
imgs = glob.glob(os.path.join(output_folder, "*00.png"))
imgs.sort()
imgs = imgs[1:]
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)
# Save into a GIF file that loops forever
duration = 150
frames[0].save(os.path.join(output_folder, f'simulated_TC{duration}.gif'), format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=duration, loop=0)
new_frame.close()

In [ ]:
hv.save?